# Text generation with an RNN

## Setup

### Import TensorFlow and other libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

### Read the data

First, look in the text:

In [6]:
# Read, then decode for py2 compat.
text = open('complete_seinfeld_scripts.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 4351404 characters


In [11]:
# Take a look at the first 250 characters in text
print(text[:1500])


JERRY: You know, why we're here? [he means: here in the "Comedy 
              club"] To be out, this is out...and out is one of the single 
              most enjoyable experiences of life. People...did you ever hear people 
              talking about "We should go out"? This is what they're 
              talking about...this whole thing, we're all out now, no one is home. 
              Not one person here is home, we're all out! There are people tryin' 
              to find us, they don't know where we are. [imitates one of these 
              people "tryin' to find us"; pretends his hand is a phone] 
              "Did you ring?, I can't find him." [imitates other person 
              on phone] "Where did he go?" [the first person again] 
              "He didn't tell me where he was going". He must have gone 
              out. You wanna go out: you get ready, you pick out the clothes, 
              right? You take the shower, you get all ready, get the cash, get 
         

In [12]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

104 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [13]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `preprocessing.StringLookup` layer:

In [14]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

It converts form tokens to character IDs, padding with `0`:

In [15]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[67, 68, 69, 70, 71, 72, 73], [90, 91, 92]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the padding and `[UNK]` tokens are set the same way.

In [16]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [17]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [18]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [19]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [20]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(4351404,), dtype=int64, numpy=array([ 2, 45, 40, ..., 67, 17,  3])>

In [21]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [22]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))



J
E
R
R
Y
:
 
Y
o


In [23]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [24]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\n' b'J' b'E' b'R' b'R' b'Y' b':' b' ' b'Y' b'o' b'u' b' ' b'k' b'n'
 b'o' b'w' b',' b' ' b'w' b'h' b'y' b' ' b'w' b'e' b"'" b'r' b'e' b' '
 b'h' b'e' b'r' b'e' b'?' b' ' b'[' b'h' b'e' b' ' b'm' b'e' b'a' b'n'
 b's' b':' b' ' b'h' b'e' b'r' b'e' b' ' b'i' b'n' b' ' b't' b'h' b'e'
 b' ' b'"' b'C' b'o' b'm' b'e' b'd' b'y' b' ' b'\n' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b'c' b'l' b'u' b'b'
 b'"' b']' b' ' b'T' b'o' b' ' b'b' b'e' b' ' b'o' b'u' b't' b',' b' '
 b't' b'h' b'i'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [25]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\nJERRY: You know, why we\'re here? [he means: here in the "Comedy \n              club"] To be out, thi'
b's is out...and out is one of the single \n              most enjoyable experiences of life. People...d'
b'id you ever hear people \n              talking about "We should go out"? This is what they\'re \n      '
b"        talking about...this whole thing, we're all out now, no one is home. \n              Not one p"
b"erson here is home, we're all out! There are people tryin' \n              to find us, they don't know"


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [26]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [27]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [28]:
dataset = sequences.map(split_input_target)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [29]:
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\nJERRY: You know, why we\'re here? [he means: here in the "Comedy \n              club"] To be out, th'
Target: b'JERRY: You know, why we\'re here? [he means: here in the "Comedy \n              club"] To be out, thi'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [30]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outpts one logit for each character in the vocabulary. These are the log-liklihood of each character according to the model.

In [31]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [32]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else: 
      return x

In [33]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details asee the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 106) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [35]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  27136     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  108650    
Total params: 4,074,090
Trainable params: 4,074,090
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [36]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [37]:
sampled_indices

array([ 18,  56,  53,  20,   7,  28,  73,  93,  20,  27,  47,  99,  25,
        45,  78,  84,  94,  45,  91,  58,  59,  45,   1,  39,  97,  93,
        40,  42,  98,  23,  22,  63,  90,   8,  59,  44,   0,  11,  18,
        57,  63,  74,  50,  13, 103,  28,  96,  35,  55,  56,  31,  59,
        10,  52,  73,  47,  65,  88,  73,   3,  69,  79, 104, 101,   9,
        76,  26,  42,  55,  35,  97,  32,  73,  30,  56,  86,  49,  56,
         5,  45,  62,  30,  38,  21,  97,  35,  17,  34,  72,  50,  78,
        58,  10,  48,  37,  59,  73, 101,  17,  69])

Decode these to see the text predicted by this untrained model:

In [38]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"ot just looking at it.Elaine: Uhh. This. It's like a red dot.George: What red dot? What are you talk"

Next Char Predictions:
 b'/UR1$9g{18L\xc2\x916Jlr|JyWXJ[UNK]D\xc2\x82{EG\xc2\x8543\\x%XI(/V\\hO*\xc2\x969~@TU<X\'QgL_vg cm\xcc\x80\xc2\x93&j7GT@\xc2\x82=g;UtNU"J[;C2\xc2\x82@.?fOlW\'MBXg\xc2\x93.c'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [39]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [40]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 106)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.663459


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [41]:
tf.exp(mean_loss).numpy()

106.00209

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [42]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [43]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [48]:
EPOCHS = 10

In [49]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
673/673 [==============================] - 2744s 4s/step - loss: 1.7813
Epoch 2/10
673/673 [==============================] - 2663s 4s/step - loss: 1.2831
Epoch 3/10
673/673 [==============================] - 2662s 4s/step - loss: 1.1618
Epoch 4/10
673/673 [==============================] - 2662s 4s/step - loss: 1.0974
Epoch 5/10
673/673 [==============================] - 2663s 4s/step - loss: 1.0516
Epoch 6/10
673/673 [==============================] - 2661s 4s/step - loss: 1.0141
Epoch 7/10
673/673 [==============================] - 2661s 4s/step - loss: 0.9819
Epoch 8/10
673/673 [==============================] - 2662s 4s/step - loss: 0.9530
Epoch 9/10
673/673 [==============================] - 2663s 4s/step - loss: 0.9275
Epoch 10/10
673/673 [==============================] - 2667s 4s/step - loss: 0.9054


## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [50]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    
    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [51]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [52]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
ROMEO: (shrimp) Here. You one steak in your money, he slams 
      to one yer, too.GEORGE: Well, we sees the battery place.ELAINE: No no -sight and matter?GEORGE: He's giving it! This is gonna take a whole boxe of your block![Scene ends is head to the dintany weed-the photognish, walks away. Elaine is discount and and feels EllenMs to take a hooker.Jerry: What's the drink of the rest officing?Woma

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [53]:
start = time.time()
states = None
next_char = tf.constant(['JERRY:', 'ELAINE:', 'GEORGE:', 'KRAMER:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result, '\n\n' + '_'*80)


print(f"\nRun time: {end - start}")

tf.Tensor(
[b'ROMEO: No, no. We\'ll be black."\n              Jerry: "Oh. Thanks, this is fantastic.                               fifty bucks a toothbrusk.\n\n              ----------------------------------------------------------------- \n              \n\n\n        [End Acendlanne                 sa hotel and books, he mentales. Elaine is styling a jar."\n              Mancelous: "Jerry?!"\n              George: "I think they are pretty good day."\n              Darryl: "You were the biggers?! Gimme a briefcasy.\n\n        Elaine: Jerry.\n\n        Newman returns a Jerry\'s, George does on his rain, and she wipes up inside from the aturcric masmetial\n        club confusnal.\n\n        George: Hey.\n\n        Puddy walking up the sales associate: I don\'t remember better!\n\n        Jerry: I say, I\'m an awful duents. Several recourtengure.\n\n        Elaine: Fine, about this, guess,\n        you eat prestrations. George: I really don\'t know if I had just to see how \n            

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [44]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [45]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
The sun and bid his mortal back again
To search, and keep we shall edward thee
From this most study


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [46]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [47]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [48]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [49]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 15s 76ms/step - loss: 2.7008


Or if you need more control, you can write your own complete custom training loop:

In [50]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch + 1, batch_n, logs['loss']))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print('Epoch {} Loss: {:.4f}'.format(epoch + 1, mean.result().numpy()))
    print('Time taken for 1 epoch {} sec'.format(time.time() - start))
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1587090492248535
Epoch 1 Batch 50 Loss 2.015890121459961
Epoch 1 Batch 100 Loss 1.8815938234329224
Epoch 1 Batch 150 Loss 1.8182443380355835

Epoch 1 Loss: 1.9731
Time taken for 1 epoch 14.837902069091797 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.801224946975708
Epoch 2 Batch 50 Loss 1.7420837879180908
Epoch 2 Batch 100 Loss 1.6781731843948364
Epoch 2 Batch 150 Loss 1.6495245695114136

Epoch 2 Loss: 1.6969
Time taken for 1 epoch 14.266141414642334 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6056054830551147
Epoch 3 Batch 50 Loss 1.557340145111084
Epoch 3 Batch 100 Loss 1.476023554801941
Epoch 3 Batch 150 Loss 1.5442909002304077

Epoch 3 Loss: 1.5393
Time taken for 1 epoch 14.16746735572815 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4498177766799927
Epoch 4 Batch 50 Loss 1